In [1]:
from nltk.corpus import stopwords
from collections import defaultdict
import string
import heapq
import spacy
import neuralcoref
import urllib.request
import fitz

In [3]:
nlp=spacy.load("en_core_web_md")
neuralcoref.add_to_pipe(nlp)

In [4]:
def extract_text_url(url):
    
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'


    headers={'User-Agent':user_agent} 

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    text_extracted = ''
    text = fitz.open(stream=data, filetype="pdf")
    total_page_count = text.pageCount

    for page_num in range(total_page_count):
        page_ = text.loadPage(page_num).getText()
        page_ = page_.replace('\n', ' ')
        text_extracted += page_ + ' '
    
    return text_extracted

In [5]:
text=extract_text_url("http://www.int.washington.edu/REU/2019/Chamberlain_Paper.pdf")

In [6]:
def tokenizer(text):
    sentence,word_sent=[],[]
    stopWords=set(stopwords.words("english"))
    doc=nlp(text)
    for sent in doc.sents:
        sentence.append(sent)
        words=[]
        for token in sent :
            if token.text not in stopWords or token.text not in string.punctuation:
                if token.pos_=="PRON" and token._.in_coref:
                    for cluster in token._.coref_clusters:
                        words.append((cluster.main.text,"NOUN"))
                if token.pos_=="NOUN" or token.pos_=="PROPN":
                    words.append((token.text,token.pos_))
        word_sent.append(words)
    return sentence,word_sent

In [7]:
def graph_building(word_sent):
    graph=defaultdict(int)
    for words in word_sent:
        for i in range(len(words)):
            for j in range(i+1,len(words)):
                graph[(words[i],words[j])]+=abs(i-j)
    for key,values in graph.items():
        graph[key]=1/(1+values)
    return graph

In [8]:
def sentence_extraction(graph,word_sent,sentence,gamma=0.1):
    noun_score=defaultdict(int)
    for key,value in graph.items():
        noun_score[key[0][0]]+=value
    sent_score=defaultdict(int)
    for i,words in enumerate(word_sent):
        for word in words:
            sent_score[sentence[i].text]+=(1-gamma)*noun_score[word[0]]
    summary_sent=heapq.nlargest(7,sent_score,key=sent_score.get)
    print("\n".join(summary_sent))

In [9]:
sentence,word_sent=tokenizer(text)

In [10]:
graph=graph_building(word_sent)

In [11]:
sentence_extraction(graph,word_sent,sentence)

4 Rabi Oscillation Curves Producing Rabi oscillations by altering the decoherence rate of the NV center’s spin state is vital in order to measure the π pulse time of the NV center.
If this were a Pulsed ODMR scan taken from a single NV sample, the next step would be to compare this result with the PLE taken of the same NV center to see if there is a correlation between this result and the amount of spectral diffusion that was measured for that NV center.
This ability to resolve the spin state of a single NV center yields the ability to study the hyperﬁne interaction of its spin state, as well as match the number of nuclear interactions with the respective spins of the NV center’s isotope.
After the isotope identiﬁcations are compared with their corresponding PLE spectra,the consequences of this possible relationship will result in a better understanding of not only whether ion implantation produces reliable NV centers, but will also result in a better understanding of how to potentiall